In [1]:
import torch
from torch import nn
from torch.optim import Optimizer
from tqdm import tqdm

In [83]:
class ClsHead(nn.Module):
    def __init__(self):
        super(ClsHead, self).__init__()
        # 假设你的模型有一些层
        self.layer1 = nn.Linear(32000, 128)
        self.layer2 = nn.Linear(128, 1)  # 最终输出一个值
        # self.mp = nn.AdaptiveMaxPool1d(1)

    def forward(self, x, batch_size):
        # x 的形状是 [1, batch_size*?, 32000]
        out = self.layer1(x)  # 形状变为 [1, batch_size*?, 128]
        out = self.layer2(out)  # 形状变为 [1, batch_size*?]
        out = out.view(1, -1)
        # out = self.mp(out).squeeze(2)
        
        return out

In [70]:
a = torch.randn(1, 2*155, 32000).cuda()

In [84]:
sg = ClsHead().cuda()

In [85]:
out = sg(a, 2)
label = torch.tensor([[0.0000, 0.5000]]).cuda()

In [86]:
nn.CrossEntropyLoss()(torch.sigmoid(out),label)

RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [49]:
(torch.sigmoid(out).round() == label).float().mean()

tensor(0.5000, device='cuda:0')

In [78]:
torch.sigmoid(out)

tensor([[0.6795, 0.6795]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [54]:
label

tensor([[0.0000, 0.5000]], device='cuda:0')

In [68]:
label.float()

tensor([[0.0000, 0.5000]], device='cuda:0')

In [88]:
packed_seq_lens = [154,156]
max_values = []
# 计算每个序列的起始和结束索引
current_idx = 0
for seq_len in packed_seq_lens:
    # 提取当前序列的切片
    seq = out[:, current_idx:current_idx+seq_len]
    
    # 对当前序列取最大值
    max_value = torch.max(seq, dim=1)[0]
    
    # 将最大值添加到列表中
    max_values.append(max_value)
    
    # 更新下一个序列的起始索引
    current_idx += seq_len
# 将最大值列表合并为一个新的张量
out = torch.stack(max_values, dim=1)

In [89]:
out

tensor([[0.7864, 0.9822]], device='cuda:0', grad_fn=<StackBackward0>)